In [1]:
%matplotlib inline
import requests # To read html text
from bs4 import BeautifulSoup # To parse html text
import re # Regular expressions
import operator
import matplotlib.pyplot as plt
from selenium import webdriver
import datetime

In [2]:
def resolution_filter(string):
    #string=u'\u0110\u1ed9 ph\xe2n gi\u1ea3i320 x 240 Pixels'
    result=re.finditer('[0-9]+ x [0-9]+',string)
    for ele in result:
        return ele.group()

def camera_filter(string):
    #string='16 MP và 8 MP'
    ite=re.finditer('[0-9.]+ MP',string)
    for ele in ite:
        return ele.group()
    
def width_screen_filter(string):
    result=re.search('([0-9]+.[0-9]+)|([0-9]+)"',string)
    return result.group()
def CPU_filter(string):
    ite=re.finditer('[0-9.]+ GHz',string)
    for ele in ite:
        return ele.group()
def memory_filter(string):
    ite=re.finditer('[0-9.]+ (GB|MB)',string)
    for ele in ite:
        return ele.group()
def battery_filter(string):
    ite=re.finditer('[0-9]+ mAh',string)
    for ele in ite:
        return ele.group()
def name_filter(string):
    return re.sub('<([a-z]+|/|[0-9]+)>','',string)
def price_filter(string):
    return re.sub('[^0-9.]','',string)

#print width_screen_filter('S LCD, 5.4" ')

In [3]:
def date2days(date):
    date=date.strip()
    if u'tháng' in date:
        return int(date.split()[0])*30
    if u'/' in date:
        date=date.split('/')
        date=datetime.date(int(date[2]),int(date[1]),int(date[0]))
        today=datetime.date.today() 
        return (today-date).days
    if u'năm' in date:
        return int(date.split()[0])*365

In [4]:
def get_last_page(url):#get last page of comment section
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    comment_page=tree.find_all('div',class_='pagcomment')
    try:
        last_page=(comment_page[len(comment_page)-1]('a')[-2].text)
    except:
        return '0'
    return last_page
def get_num_day(url):
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    driver.get(url)
    last_page=get_last_page(url)
    if last_page=='0':
        return '0'
    while tree.find('span',class_='active').text!=last_page:#check if it's clicked in cast of poor connection
        try:
            driver.find_element_by_link_text(last_page).click()
            tree = BeautifulSoup(driver.page_source)
            #print tree.find('span',class_='active').text
        except:
            break
    #get date
    for i in range(3):
        tree = BeautifulSoup(driver.page_source)
    date=tree.find('a',class_='time')
    try:
        return date.text
    except:
        return None
def get_confi(url):
    #input: a link
    #output: a list of configurations of a phone
    

    #proceed to the last page of comment section
    driver.get(url)
    
    #get html which contains click-button to show detailed configuration of the phone
    #try if it has ordinary form
    try :
        driver.find_element_by_class_name('viewparameterfull').click()
    except:
        return
    #let's parse some details
    tree = BeautifulSoup(driver.page_source)
    confi=tree.find(class_="parameterfull")
    resolution=confi.find(class_="g78")
    width_screen=confi.find(class_="g79")
    rear_camera=confi.find(class_="g27")
    front_camera=confi.find(class_="g29")
    CPU_speed=confi.find(class_="g51")
    RAM=confi.find(class_="g50")
    internal_storage=confi.find(class_="g49")
    battery_capacity=confi.find(class_="g84")
    while resolution==None:# DONT KNOW WHY!! If it hasn't got the detail, just keep getting it
        #let's parse some details
        try:
            driver.find_element_by_class_name('viewparameterfull').click()
        except:
            pass
        tree = BeautifulSoup(driver.page_source)
        confi=tree.find(class_="parameterfull")
        resolution=confi.find(class_="g78")
        width_screen=confi.find(class_="g79")
        rear_camera=confi.find(class_="g27")
        front_camera=confi.find(class_="g29")
        CPU_speed=confi.find(class_="g51")
        RAM=confi.find(class_="g50")
        internal_storage=confi.find(class_="g49")
        battery_capacity=confi.find(class_="g84")
    num_day=get_num_day(url)
    num_day=date2days(num_day)
    
    return [resolution_filter(resolution.text),
            width_screen_filter(width_screen.text),
            camera_filter(rear_camera.text),
            camera_filter(front_camera.text),
            CPU_filter(CPU_speed.text),
            memory_filter(RAM.text),
            memory_filter(internal_storage.text),
            battery_filter(battery_capacity.text),
            num_day]
#driver=webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
#get_confi('https://www.thegioididong.com/dtdd/nokia-222-dual-sim')

need to be altered

In [ ]:
def get_phone_info(url):#get phone info based on Brand consist of brands, prices and configurations
    #input: a link
    #output: a list of details of a single phone
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    count=0
    
    phone_list=tree.find(class_='cate filter-cate')
    prices=phone_list.find_all('strong')
    names=phone_list.find_all('h3')
    href=[ele['href'] for ele in phone_list.find_all('a')]
    brands=[]#contains a set of brand
    details=[]#contains everything
    
    for i in range(len(prices)):
        if u'Giá online' in prices[i].text:#eleminate GiaOnline and retain GiaThuong
            continue
        brands.append(names[count].text.split()[0])
        confi=get_confi('https://www.thegioididong.com/' + href[count])
        temp=[names[count].text]#,prices[i].text]
        
        if confi!=None:################################################################# SOLVE SPECIAL CASES BEFORE DELETING DIS
            temp.extend(confi)
            temp.append(price_filter(prices[i].text))
        print temp
        details.append(temp)
        count+=1
        
    return details
#driver=webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
#get_phone_info('https://www.thegioididong.com/dtdd-apple-iphone')

def list_running(url):
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    phone_list=tree.find(class_='cate filter-cate')
    links=phone_list.find_all('a')#get tags that contain links
    for ele in links:
        print ele['href']
        get_phone_info('https://www.thegioididong.com' + ele['href'])
#driver=webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
#list_running('https://www.thegioididong.com/dtdd-apple-iphone')

In [ ]:
# Read html text and parse it
url = 'https://www.thegioididong.com/dtdd'
html_text = requests.get(url).text
tree = BeautifulSoup(html_text)
driver = webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
menu = tree.find(class_='manufacture')
side=menu.find_all('aside')
details=[]
for categories in side:
    categories= categories.find_all('a')
    #print categories
    for category in categories:
        details.extend(get_phone_info('https://www.thegioididong.com' + category['href']))
print 'DONE'

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


[u'iPhone 7 Plus 256GB', u'1080 x 1920', u'5.5', u'12 MP', u'7 MP', u'2.3 GHz', u'3 GB', u'256 GB', u'2900 mAh', 240, u'25.990.000']
[u'iPhone 7 Plus Red 128GB', u'1080 x 1920', u'5.5', u'12 MP', u'7 MP', u'2.3 GHz', u'3 GB', u'128 GB', u'2900 mAh', 90, u'23.990.000']
[u'iPhone 7 Plus 128GB', u'1080 x 1920', u'5.5', u'12 MP', u'7 MP', u'2.3 GHz', u'3 GB', u'128 GB', u'2900 mAh', 240, u'23.990.000']
[u'iPhone 7 Plus 32GB', u'1080 x 1920', u'5.5', u'12 MP', u'7 MP', u'2.3 GHz', u'3 GB', u'32 GB', u'2900 mAh', 365, u'21.990.000']
[u'iPhone 7 256GB', u'1334 x 750', u'4.7', u'12 MP', u'7 MP', u'2.3 GHz', u'2 GB', u'256 GB', u'1960 mAh', 210, u'21.990.000']
[u'iPhone 7 128GB', u'1334 x 750', u'4.7', u'12 MP', u'7 MP', u'2.3 GHz', u'2 GB', u'128 GB', u'1960 mAh', 240, u'19.990.000']
[u'iPhone 7 Red 128GB', u'1334 x 750', u'4.7', u'12 MP', u'7 MP', u'2.3 GHz', u'2 GB', u'128 GB', u'1960 mAh', 90, u'19.990.000']
[u'iPhone 7 32GB', u'1334 x 750', u'4.7', u'12 MP', u'7 MP', u'2.3 GHz', u'2 GB', u

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(details, columns=['Phone Name','resolution','screen size','rear camera','front camera','CPU', 'RAM', 'Internal storage','Battery capacity','Days','Price'])

In [ ]:
df.to_csv('thegioididong2.csv', encoding='utf-8')

In [ ]:
df

# Get date

In [ ]:
def test(url):
    html_text = requests.get(url).text
    tree = BeautifulSoup(html_text)
    comment_page=tree.find(class_='characteristics')('h2')
    for ele in comment_page('h2'):
        print ele
    '''driver=webdriver.Chrome("F:\\Deadlines\\chromedriver.exe")
    driver.get(url)
    
    while tree.find('span',class_='active').text!=last_page:#check if it's clicked in cast of poor connection
        try:
            driver.find_element_by_link_text(last_page).click()
            tree = BeautifulSoup(driver.page_source)
            print tree.find('span',class_='active').text
        except:
            break
    #get date
    for i in range(3):
        tree = BeautifulSoup(driver.page_source)
    date=tree.find('a',class_='time')
    print date.text'''
test('https://www.thegioididong.com/dtdd/oppo-f3')

In [ ]:
type(driver.page_source)

In [ ]:
import datetime
def date2days(date):
    date=date.strip()
    
    if u'tháng' in date:
        return int(date.split()[0])*30
    if u'/' in date:
        date=date.split('/')
        date=datetime.date(int(date[2]),int(date[1]),int(date[0]))
        today=datetime.date.today() 
        return (today-date).days
    if u'năm' in date:
        return int(date.split()[0])*365
print get_num_day(u' 2 năm')

# regular expression test

In [ ]:
url = 'https://www.thegioididong.com/dtdd'
html_text = requests.get(url).text
tree = BeautifulSoup(html_text)

menus = tree.find_all(class_='feature')
for menu in menus:
    print menu.find('a')['href']

In [ ]:
string='Dài 158.2 mm - Ngang 77.9 mm - Dày 7.3 mm'
#ite=re.finditer('([0-9.]+) mm',string)#.group()
ite=re.finditer('[0-9.]+',string)
for ele in ite:
    print ele.group()

In [ ]:
string='<h>Asus Zenfone Live ZB501KL</> <3>'
result=re.sub('<([a-z]+|/|[0-9]+)>','',string)
print result

In [ ]:
string=u'Gi\xe1 th\u01b0\u1eddng: 1.690.000\u20ab'
result=re.sub('[^0-9.]','',string)
print result

In [ ]:
string=u'\u0110\u1ed9 ph\xe2n gi\u1ea3i320 x 240 Pixels'
result=re.finditer('[0-9]+ x [0-9]+',string)
for ele in result:
    print ele.group()

In [ ]:
string='16 MP và 8 MP'
ite=re.finditer('[0-9]+ MP',string)
for ele in ite:
    print ele.group()